<h2>Demo av folium/leaflet</h2>

Aktiver cellene under, og start et kartsøk til slutt.

In [11]:
from collections import Counter
import folium
from folium.plugins import MarkerCluster
import dhlab.nbtext as nb
import requests
from bs4 import BeautifulSoup

In [53]:
def finnsted(sted):
    r = requests.get("https://ws.geonorge.no/SKWS3Index/ssr/sok", params = {'navn': sted})
    return r.text

def koord(sted, navnetype = 'By'):
    soup = BeautifulSoup(finnsted(sted),'xml')
    res = []
    for n in soup.find_all('stedsnavn'):
        #print(n)
        if n.navnetype != None and n.navnetype.text == navnetype:
            res.append([n.stedsnavn.text, n.aust.text, n.nord.text, n.epsgKode.text])
    return res
    

def enheter(sted, limit=50):
    enheter = ('By','Dal','Bydel','Elv', 'Tettsted','Fjell', 
               'Fjellområde','Fjord','Flyplass','Foss','Hotell','Innsjø','Kanal', 'Kirke',
                'Kommune','Øy','Skog', 'Strand','Sykehus')
    
    rows = sqlcon(geocon, "select enh_snavn ||', '||enh_navntype || ' i ' || kode, ST_AsLatLonText(wkb_geometry, 'D.DDDD') from \
    stedsinfo where enh_snavn ilike %s and enh_navntype in %s limit %s", (sted, enheter, limit))
    coords = []
    for row in rows:
        x = row[1].split()
        #print(x)
        coords.append([row[0], x])
    return coords


def byer(sted, limit=50):
    rows = sqlcon(geocon, "select kode ||', '||enh_navntype , ST_AsLatLonText( wkb_geometry, 'D M') from \
    stedsinfo where kode ilike %s limit %s", (sted, limit))
    coords = []
    for row in rows:
        x = row[1].split()
        #print(x)
        coords.append([row[0], [int(x[0]) + int(x[1])/100, int(x[2]) + int(x[3])/100]])
    return coords

def kart(sted, limit=50):
    import numpy as np
    coords = koord(sted)
    kartsnitt = folium.Map(location=[62, 4], zoom_start = 4)
    if coords != []:
        locs = []
        pops = []
        #beregn gjennomsnittlig lengde og bredde og start kartet derfra
        lat = np.mean([float(crd[1][0]) for crd in coords])
        lon = np.mean([float(crd[1][1]) for crd in coords])
        kartsnitt = folium.Map(location=[lat, lon], zoom_start = 6)
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_children(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

def leggtil(kartsnitt, sted, limit=50):
    import numpy as np
    coords = enheter(sted, int(limit/2))
    if coords != []:
        locs = []
        pops = []
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_children(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

In [51]:
#!pip install convertbng
!pip install utm

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import utm
utm.to_latlon(340000, 5710000, 32, 'U')
(51.51852098408468, 6.693872395145327)

(51.51852098408468, 6.693872395145327)

In [9]:
from convertbng.util import convert_bng, convert_lonlat,convert_etrs89_to_lonlat,convert_epsg3857_to_wgs84


In [63]:
for x in koord('kristiansand'):
    e = float(x[1])
    n = float(x[2])
    print(e, n)
    (a, b) = (utm.to_latlon(e, n, 32,  'u'))
    print((a,b))


88128.04 6466407.01


OutOfRangeError: easting out of range (must be between 100.000 m and 999.999 m)

In [55]:
koord("tromsø", "By")

[['Tromsø', '653706.32', '7732337.59', '25833']]

In [27]:
nb.xmlpretty(finnsted('tromsø'))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<html>
 <body>
  <sokres>
   <sokstatus>
    <ok>
     true
    </ok>
    <melding>
    </melding>
   </sokstatus>
   <totaltantalltreff>
    15
   </totaltantalltreff>
   <stedsnavn>
    <ssrid>
     306798
    </ssrid>
    <navnetype>
     By
    </navnetype>
    <kommunenavn>
     Tromsø
    </kommunenavn>
    <fylkesnavn>
     Troms
    </fylkesnavn>
    <stedsnavn>
     Tromsø
    </stedsnavn>
    <aust>
     653706.32
    </aust>
    <nord>
     7732337.59
    </nord>
    <skrivemaatestatus>
     Godkjent
    </skrivemaatestatus>
    <spraak>
     NO
    </spraak>
    <skrivemaatenavn>
     Tromsø
    </skrivemaatenavn>
    <epsgkode>
     25833
    </epsgkode>
   </stedsnavn>
   <stedsnavn>
    <ssrid>
     735562
    </ssrid>
    <navnetype>
     Lykt (Fyrlykt)
    </navnetype>
    <kommunenavn>
     Tromsø
    </kommunenavn>
    <fylkesnavn>
     Troms
    </fylkesnavn>
    <stedsnavn>
     Tromsø bru
    </stedsnavn>
   